Librerias

In [ ]:
import numpy as np
from scipy.stats import kappa3
from scipy.stats import kappa4
from scipy.stats import gibrat
import matplotlib.pyplot as plt

Definición de constantes:

In [ ]:
HV = 99999999999999999999999999999999

Inicalizar variables de control

In [ ]:
#Variables de control
N = 6
F = 999999

#Fin de la simulación
TF = 390000 #Equivale a 9 meses, en minutos

#======SLA==========
SLA_HORAS = 48 #horas
SLA_HORAS_DIA = SLA_HORAS / 3
SLA = SLA_HORAS_DIA * 60

Inicializar tabla de eventos futuros y variables de resultado y de estado

In [ ]:
#TEF
TPLL = 0
TPSF = HV
TPSE = []

#Resultado
PTO = []
PPS = 0
PTF = 0

#Estado
ns = 0

#Auxiliares
nt = 0
ITO = []
T = 0
STO = []
ia = 0
ta = 0
x = 0
nf = 0
STLL = 0
STS = 0
STF = 0
ITF = 0

#############
p = 0
while p<N:
  TPSE.append(HV)
  PTO.append(HV)
  STO.append(T)
  ITO.append(T)
  p = p + 1


Subrutinas - FDP

In [ ]:
def ia():
  temp = kappa4.rvs(h=0.7882610005908149, k=-1.1313667389574324, loc=7.047753953250408, scale=34.21147541897436,size=1)[0]
  if temp > 4000:
    temp = kappa4.rvs(h=0.7882610005908149, k=-1.1313667389574324, loc=7.047753953250408, scale=34.21147541897436,size=1)[0]
  return (temp)

def ta():
  return (gibrat.rvs(loc=-91.13816963997854, scale=350.7249689146642,size=1)[0])
  #return (kappa3.rvs(a=0.7005864105474631, loc=1.3499999986721618, scale=1153.2454727024315,size=1)[0])


Otras subrutinas

In [ ]:
def obtenerMenorTPSE():
  q = 0
  j = 1
  while j<N:
    if TPSE[q]>TPSE[j]:
      q = j
    j = j+1
  return(q)

def buscarDesocupado():
  j = 0
  while j<N:
    if TPSE[j]==HV:
      break
    j = j+1
  return(j)



Simulación

In [ ]:
noVolverAEntrar = True
while (T<TF or (T>=TF and ns!=0)):
  x = obtenerMenorTPSE()
  if(TPLL<=TPSE[x]):
    #LLEGADA
    T=TPLL
    STLL = STLL + T
    TPLL = T + ia()
    ns = ns + 1
    nt = nt + 1
    if ((ns<=N) or (ns==N+1 and TPSF != HV)):
      y = buscarDesocupado()
      TPSE[y] = T + ta()
      STO[y] = STO[y] + T - ITO[y]
    elif (ns==F and TPSF==HV):
      TPSF = T + ta()
      ITF = T

  elif (TPSE[x]<=TPSF):
    #SALIDAE
    T = TPSE[x]
    STS = STS + T
    ns = ns - 1
    if ((ns>=N+1) or (ns>=N and TPSF==HV)):
      TPSE[x] = T + ta()
    else:
      TPSE[x] = HV
      ITO[x] = T

  else:
    #SALIDAF
    T = TPSF
    STF = STF + T - ITF
    STS = STS + T
    ns = ns - 1
    nf = nf + 1
    if (ns>=F):
      TPSF = T + ta()
      ITF = T
    else:
      TPSF = HV

  if (T>=TF and noVolverAEntrar): #Condición de vaciamiento
    TPLL = HV
    noVolverAEntrar = False
    print("Datos al momento de comenzar el vaciamiento")
    print("Cola:", ns)
    print("Valor de T:", T)



#Impresión de resultados
print("")
print("")
print("=== Parámetros de la simulación ===")
print("Duración de la simulación: ", T, " Tiempo final seteado: ", TF)
print("N:", N)
print("F:", F)
print("SLA:", SLA)
print("=== Resultados de la simulación ===")
print("Porcentaje trabajado por el Freelance (PTF):", STF*100/T,"%", "STF (Es concurrente):", STF)
print("Promedio de permanencia en el sistema (PPS):", (STS-STLL)/nt)
print("Porcentaje de tickets resueltos por el freelance (PRF):",  nf*100/nt, "%")

print("Porcentaje de tiempo ocioso")
x = 0
while x<N:
  print(">",x+1,": ", STO[x]*100/T, "%")
  x = x+1


print("Tickets totales:", nt, "Tickets atendidos por Freelance: ", nf)

Datos al momento de comenzar el vaciamiento
Cola: 7
Valor de T: 390001.5447895366


=== Parámetros de la simulación ===
Duración de la simulación:  391466.79173082457  Tiempo final seteado:  390000
N: 6
F: 999999
SLA: 960.0
=== Resultados de la simulación ===
Porcentaje trabajado por el Freelance (PTF): 0.0 % STF (Es concurrente): 0
Promedio de permanencia en el sistema (PPS): 677.0429741840406
Porcentaje de tickets resueltos por el freelance (PRF): 0.0 %
Porcentaje de tiempo ocioso
> 1 :  39.27271208295467 %
> 2 :  39.916617248874566 %
> 3 :  42.89034586396478 %
> 4 :  50.23591382001411 %
> 5 :  50.04863511066026 %
> 6 :  58.37116023213142 %
Tickets totales: 2606 Tickets atendidos por Freelance:  0
